In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import shapely
from shapely.geometry import shape, mapping
import geopandas as gpd
from geopandas.tools import sjoin
import requests 
import json
import csv
from kmodes.kprototypes import KPrototypes
from kmodes.kmodes import KModes

import warnings
warnings.filterwarnings("ignore")
import os


In [22]:
import sys
!{sys.executable} -m pip install kmodes

In [3]:
Ballot_1_2014_summary = pd.read_csv("14_1_merge_sum.csv")
Ballot_1_2014_summary

,Locality,Ward,Pct,Total:Population of one race:White,Total:Population of one race:Black or African American,Total:Population of one race:American Indian and Alaska Native,Total:Population of one race:Asian,Total:Population of one race:Native Hawaiian and Other Pacific Islander,Total:Population of one race:Some other race,Total:Population of two or more races:,Total:Population of two or more races:Two races including Some other race,"Total:Population of two or more races:Two races excluding Some other race, and three or more races",Total:Population of two or more races:Population of two races:,Total:Population of two or more races:Population of two races:White; Black or African American,Total:Population of two or more races:Population of two races:White; American Indian and Alaska Native,Total:Population of two or more races:Population of two races:White; Asian,Total:Population of two or more races:Population of two races:Black or African American; American Indian and Alaska Native,Total:Population of two or more races:Population of two races:All other two race combinations,Total:Population of two or more races:Population of three races,Total:Population of two or more races:Population of four or more races
0,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,1733,3389,5050,1799,544,790,25,1892,72,0
1,ABINGTON,-,2,158272,2252,0,3545,8,2208,3616,475,3141,3504,1628,584,817,0,475,30,82
2,ABINGTON,-,3,104670,2394,63,3425,8,1812,2043,176,1867,1903,881,318,480,0,224,58,82
3,ABINGTON,-,4,106625,9663,113,3411,0,3707,3368,1072,2296,3270,1046,422,639,16,1147,98,0
4,ABINGTON,-,5,77872,1298,0,2088,0,1180,2100,312,1788,2100,976,320,492,0,312,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,WOBURN,6,2,13078,1008,0,3996,0,156,630,0,630,630,174,0,424,0,32,0,0
1425,WOBURN,7,1,17781,752,0,1477,0,310,401,45,356,299,24,16,214,0,45,80,22
1426,WOBURN,7,2,19887,597,0,1986,0,420,393,45,348,393,36,24,288,0,45,0,0
1427,WORTHINGTON,-,1,79542,1617,118,2445,0,308,1672,126,1546,1580,442,355,581,18,184,92,0


In [30]:
# First we will keep a copy of data
Ballot_1_2014_summary_copy = Ballot_1_2014_summary.copy()

In [29]:
km_cao = KModes(n_clusters=2, init = "Cao", n_init = 1, verbose=1)
fitClusters_cao = km_cao.fit_predict(Ballot_1_2014_summary)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 22, cost: 22042.0


In [31]:
clusterCentroidsDf = pd.DataFrame(km_cao.cluster_centroids_)
clusterCentroidsDf.columns = Ballot_1_2014_summary.columns
# Mode of the clusters
clusterCentroidsDf

,Locality,Ward,Pct,Total:Population of one race:White,Total:Population of one race:Black or African American,Total:Population of one race:American Indian and Alaska Native,Total:Population of one race:Asian,Total:Population of one race:Native Hawaiian and Other Pacific Islander,Total:Population of one race:Some other race,Total:Population of two or more races:,Total:Population of two or more races:Two races including Some other race,"Total:Population of two or more races:Two races excluding Some other race, and three or more races",Total:Population of two or more races:Population of two races:,Total:Population of two or more races:Population of two races:White; Black or African American,Total:Population of two or more races:Population of two races:White; American Indian and Alaska Native,Total:Population of two or more races:Population of two races:White; Asian,Total:Population of two or more races:Population of two races:Black or African American; American Indian and Alaska Native,Total:Population of two or more races:Population of two races:All other two race combinations,Total:Population of two or more races:Population of three races,Total:Population of two or more races:Population of four or more races
0,BOSTON,-,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,LYNN,-,2,13368,173,0,233,0,36,338,15,227,338,75,42,70,0,104,0,0


In [ ]:
Ballot_1_2014_summary = Ballot_1_2014_summary_copy.reset_index()

In [7]:
Ballot_1_2014 = pd.read_csv("PD43+__2014_Statewide_Question_1Do_you_approve_of_a_law_summarized_below_on_w_including_precincts_merge.csv")
Ballot_1_2014_subset = Ballot_1_2014[['Locality','Ward','Pct','Yes','No','Blanks','Total Votes Cast']]
Ballot_1_2014_subset


/Users/gowthamasokan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (28,65,66,67) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Locality,Ward,Pct,Yes,No,Blanks,Total Votes Cast
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
74329,LINCOLN,-,2,398,608,32.0,"1,038"
74330,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74331,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74332,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
Ballot_1_2014_subset.rename(columns={'Locality': 'Locality_Vote', 'Ward': 'Ward_Vote', 'Pct': 'Pct_vote'}, inplace=True)



/Users/gowthamasokan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [20]:
Locality_list = []
Ward_list = []
Pct_list = []

for locality, ward, pct in zip(list(Ballot_1_2014_subset['Locality_Vote'].unique()), list(Ballot_1_2014_subset['Ward_Vote'].unique()), list(Ballot_1_2014_subset['Pct_vote'].unique())):
    for locality_sum, ward_sum, pct_sum in zip(Ballot_1_2014_summary['Locality'], Ballot_1_2014_summary['Ward'], Ballot_1_2014_summary['Pct']):
        if (locality_sum == locality) & (ward_sum == ward) & (pct_sum == pct):
            Locality_list.append(locality)
            Ward_list.append(ward)
            Pct_list.append(pct)

                               
Ward_list    
    
    
    

['-']

In [15]:
df_2014_B1 = pd.merge(Ballot_1_2014_summary, Ballot_1_2014_subset,  how='left', left_on=['Locality','Ward','Pct'], right_on = ['Locality_Vote','Ward_Vote','Pct_vote'])
df_2014_B1





,Locality,Ward,Pct,Total:Population of one race:White,Total:Population of one race:Black or African American,Total:Population of one race:American Indian and Alaska Native,Total:Population of one race:Asian,Total:Population of one race:Native Hawaiian and Other Pacific Islander,Total:Population of one race:Some other race,Total:Population of two or more races:,...,Total:Population of two or more races:Population of two races:All other two race combinations,Total:Population of two or more races:Population of three races,Total:Population of two or more races:Population of four or more races,Locality_Vote,Ward_Vote,Pct_vote,Yes,No,Blanks,Total Votes Cast
0,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,1892,72,0,ABINGTON,-,1,710,337,21.0,"1,068"
1,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,1892,72,0,ABINGTON,-,1,710,337,21.0,"1,068"
2,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,1892,72,0,ABINGTON,-,1,710,337,21.0,"1,068"
3,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,1892,72,0,ABINGTON,-,1,710,337,21.0,"1,068"
4,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,1892,72,0,ABINGTON,-,1,710,337,21.0,"1,068"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53894,WRENTHAM,-,1,229424,9151,345,9769,15,1850,5451,...,756,446,0,WRENTHAM,-,1,878,613,56.0,"1,547"
53895,WRENTHAM,-,1,229424,9151,345,9769,15,1850,5451,...,756,446,0,WRENTHAM,-,1,878,613,56.0,"1,547"
53896,WRENTHAM,-,1,229424,9151,345,9769,15,1850,5451,...,756,446,0,WRENTHAM,-,1,878,613,56.0,"1,547"
53897,WRENTHAM,-,1,229424,9151,345,9769,15,1850,5451,...,756,446,0,WRENTHAM,-,1,878,613,56.0,"1,547"


In [11]:
df_2014_B1 = Ballot_1_2014_summary.merge(Ballot_1_2014_subset, how='inner', left_on=['Locality','Ward','Pct'], right_on=['Locality','Ward','Pct'])
df_2014_B1



,Locality,Ward,Pct,Total:Population of one race:White,Total:Population of one race:Black or African American,Total:Population of one race:American Indian and Alaska Native,Total:Population of one race:Asian,Total:Population of one race:Native Hawaiian and Other Pacific Islander,Total:Population of one race:Some other race,Total:Population of two or more races:,...,Total:Population of two or more races:Population of two races:White; American Indian and Alaska Native,Total:Population of two or more races:Population of two races:White; Asian,Total:Population of two or more races:Population of two races:Black or African American; American Indian and Alaska Native,Total:Population of two or more races:Population of two races:All other two race combinations,Total:Population of two or more races:Population of three races,Total:Population of two or more races:Population of four or more races,Yes,No,Blanks,Total Votes Cast
0,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,544,790,25,1892,72,0,710,337,21.0,"1,068"
1,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,544,790,25,1892,72,0,710,337,21.0,"1,068"
2,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,544,790,25,1892,72,0,710,337,21.0,"1,068"
3,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,544,790,25,1892,72,0,710,337,21.0,"1,068"
4,ABINGTON,-,1,130066,12649,386,3870,0,4668,5122,...,544,790,25,1892,72,0,710,337,21.0,"1,068"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53879,WRENTHAM,-,1,229424,9151,345,9769,15,1850,5451,...,306,1476,219,756,446,0,878,613,56.0,"1,547"
53880,WRENTHAM,-,1,229424,9151,345,9769,15,1850,5451,...,306,1476,219,756,446,0,878,613,56.0,"1,547"
53881,WRENTHAM,-,1,229424,9151,345,9769,15,1850,5451,...,306,1476,219,756,446,0,878,613,56.0,"1,547"
53882,WRENTHAM,-,1,229424,9151,345,9769,15,1850,5451,...,306,1476,219,756,446,0,878,613,56.0,"1,547"
